In [2]:
import cv2
import time
import pytesseract

import numpy as np

from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [16]:
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Removed headless mode
driver = webdriver.Chrome(options=options)
driver.maximize_window()

url = 'https://www.hackthissite.org/user/login'
driver.get(url)
time.sleep(2)

username_field = driver.find_element(By.ID, 'login_username')
password_field = driver.find_element(By.ID, 'login_password')
login_button = driver.find_element(By.XPATH, '/html/body/table/tbody/tr[2]/td/table/tbody/tr/td[2]/form/table/tbody/tr[4]/td/input')

username_field.send_keys('abc')
password_field.send_keys('123')
login_button.click()
time.sleep(2)

try:
    captcha_path = '/html/body/table/tbody/tr[2]/td/table/tbody/tr/td[2]/form/table/tbody/tr[5]/td/img'
    captcha_element = driver.find_element(By.XPATH, captcha_path)
    driver.save_screenshot('webpage.png')

    # crop the captcha image from the screenshot
    captcha_location = captcha_element.location
    captcha_size = captcha_element.size
    device_pixel_ratio = driver.execute_script("return window.devicePixelRatio;")
    x = int(captcha_location['x'] * device_pixel_ratio)
    y = int(captcha_location['y'] * device_pixel_ratio)
    w = int(captcha_size['width'] * device_pixel_ratio)
    h = int(captcha_size['height'] * device_pixel_ratio)

    img = Image.open('webpage.png')
    captcha_img = img.crop((x, y, x + w, y + h))
    captcha_img.save('captcha.png')

    # Convert PIL image to OpenCV format
    captcha_cv = np.array(captcha_img)
    captcha_cv = cv2.cvtColor(captcha_cv, cv2.COLOR_RGB2BGR)

    # Convert to grayscale
    gray = cv2.cvtColor(captcha_cv, cv2.COLOR_BGR2GRAY)

    # Optional: Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Adaptive Thresholding instead of Otsu (works better on uneven lighting)
    thresh = cv2.adaptiveThreshold(
        blurred, 255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 
        11, 2
    )

    # Morphological operations (first erode then dilate — opening)
    kernel = np.ones((2, 2), np.uint8)
    processed = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    # Invert the image if needed (white text on black image causes issues)
    processed = cv2.bitwise_not(processed)


    # Save result
    cv2.imwrite('processed_captcha.png', processed)

    captcha_text = pytesseract.image_to_string(Image.fromarray(processed), config='--psm 8').strip()
    print(f'Extracted Captcha Text: {captcha_text}')
    
    if captcha_text:
        username_field = driver.find_element(By.ID, 'login_username')
        password_field = driver.find_element(By.ID, 'login_password')
        login_button = driver.find_element(By.XPATH, '/html/body/table/tbody/tr[2]/td/table/tbody/tr/td[2]/form/table/tbody/tr[6]/td/input')
        captcha_input = driver.find_element(By.ID, 'login_captcha')
        
        username_field.clear()
        password_field.clear()
        
        username_field.send_keys('abc')
        password_field.send_keys('123')
        captcha_input.send_keys(captcha_text)
        
        login_button.click()
        time.sleep(4)
        
    else:
        print("Captcha text extraction failed or returned empty string.")

except Exception as e:
    pass
finally:
    time.sleep(2)
    driver.quit()

Extracted Captcha Text: ee Sa
